<br>
<a href="https://www.nvidia.cn/training">
    <div style="width: 55%; background-color: white; margin-top: 50px;">
    <img src="https://dli-lms.s3.amazonaws.com/assets/general/nvidia-logo.png"
         width="400"
         height="186"
         style="margin: 0px -25px -5px; width: 300px"/>
</a>
<h1 style="line-height: 1.4;"><font color="#76b900"><b>使用大语言模型（LLM）构建 AI 智能体</h1>
<h2><b>主题 1：</b> 试用 CrewAI</h2>
<br>


**这是几个额外主题中的第一个**，我们会讨论一些需要了解的重要内容，但在本课程中不会过多关注。请考虑一下为什么不专注于这些材料，但也要尽量理解这些内容是将来您可能会遇到的一个切入点。

在之前的 notebook 中，实际上定义了自己的一套小型智能体系统，虽然只是为了让两个非人类智能体有机会以合理的方式进行对话。本 notebook 将简单看看一个特别受欢迎、且在建模个性化智能体方面非常有用的智能体框架，并且非常容易应用上：[**CrewAI**](https://www.crewai.com/open-source)！

### **学习目标：**

**本 notebook 将:**

- 学习一些预构建的智能体框架，这些框架实现了我们早期的抽象，并进行了调整和扩展。
- 具体研究 CrewAI，考虑如何复制之前的师生对话。

<hr><br>

### 定义智能体框架

无论您是否完全理解，之前的 notebook 中展示的两个系统实际上都是**智能体系统**。它们是软件系统，至少有一个软件组件能够语义上感知环境，并尽其所能地响应以满足模糊的目标。
- 基本的聊天循环实际上就是一个循环，从 LLM 和用户中获取响应。环境是消息总线，智能体是您和 LLM，而一个过程则是从智能体流出的响应。
- 本地视角的系统，同样在循环中定义，与前者非常相似，但在建模多个不同个性方面支持得更好。它们从某个全局状态系统映射到 LLM 适合处理的内容，然后以相同的方式映射回全局空间。

从技术上讲，我们依赖于 LangChain 软件栈连接到下方的模型，经过大量抽象，但实际上我们使用的是一些简单的原语来使智能体系统工作。真正有趣的部分在于这些原语的组织，创造出协同的组件，使智能体，甚至人类，能够适当地进行交流。 

**这个 notebook 将简要介绍 CrewAI。** 尽管课程中对 CrewAI 的使用不多-稍后会解释原因-但理解以下几点是很重要的：
- CrewAI 是什么。
- CrewAI 解决了什么问题。
- 人们为什么选择使用它。

<br>

## **什么是 CrewAI?**

以下是来自于[**官方 CrewAI 文档**](https://docs.crewai.com/introduction)的直接摘录（截取于 2025 年 2 月 21 日）:

<img src="images/crewai-purpose.png" style="width: 800px"/>

<!-- > **CrewAI 是一个前沿框架，用于调度自主 AI 智能体。**
> 
> CrewAI 使您能够创建 AI 团队，每个智能体都有特定的角色、工具和目标，共同协作完成复杂任务。
>
> 想象一下，您正在组建理想的团队 - 每个成员（智能体）都带来了独特的技能和专业知识，顺畅地合作，以实现您的目标。 -->

如宣传所言，**CrewAI** 是一个构建良好的通用多智能体框架，粗略来说，它具备以下特点：
- 一个强制配对的通信机制。
- 一些核心工作流，可以帮助简化流程。
- 一些原语的定义，使工作流易于执行。
- 支持多租户（multi-tenant）和并发执行的生产化路径。(*后面会详细介绍*)

在下面的部分中，我们将利用其内置的原语，研究这个系统的运作方式，并考虑何时可以使用它。

#### **CrewAI 思维导图**

像所有框架一样，CrewAI 对智能体系统的结构/最佳支持的类型有一些看法。以下是他们框架的最新工作思维导图，供首次了解其框架的朋友参考。

<img src="images/crewai-mindmap.png" style="width: 800px"/>

每当您看到这样的内容时，都要明白这是 **一种潜在的思考方式**，来考虑智能体抽象。每个潜在选项都有其优缺点，我们认为宣传 CrewAI 重要，但在课程中不明确教授其抽象，亦有其原因。

明确来说，他们的抽象功能足以执行本课程范围内所覆盖的流程，鼓励您在课程结束后进行更深入的使用案例尝试！

#### **定义 LLM 客户端**

虽然 CrewAI 和 LangChain 之间确实有一些共享的集成和兼容层，但 CrewAI 开箱即用时更倾向于遵循与 LangChain 不同的 LLM 客户端定义。

实际上，它们都做的大致相同，但对这两个框架使用的接口实现不同。因此，接下来需要稍微不同地构造接口：

In [ ]:
from crewai import LLM

llm = LLM(
    model="nvidia_nim/meta/llama-3.1-8b-instruct",   ## Provider Class / Model Published / Model Name
    base_url="http://nim-llm:8000/v1",        ## Url to send your request to (ChatNVIDIA accepts env variable)
    temperature=0.7,
    api_key="PLACEHOLDER",                           ## API key is required by default.
)

llm.call(messages=[{"role": "user", "content": "What is the capital of France?"}])  ## Call, not "invoke"

#### **定义 "链原语"**

在 LangChain 中，运行时接口让我们能够轻松地将多个组件连接在一起，以便连接缓冲区和/或简单调用。回想一下常见的 `prompt | llm | StrOutputParser()` 链，之后将探索这些抽象的更有趣的衍生结果。

在 CrewAI 中，许多核心原语都是为特定机制而设计的，以非常明确的方式与智能体通信缓冲区进行交互。例如，以下单元展示了一个最小 CrewAI `Crew`（智能体池）的典型构造，它们共同朝着某个目标努力：

> 您可以定义一个或多个 CrewAI [**`智能体`**](https://docs.crewai.com/concepts/agents)，这些智能体基于个性，能够与其他智能体进行通信。结合 `提示词` 的实用抽象，`task_execution()` 方法为智能体提供了基础提示词（后续可以通过其它机制添加）。
> 
> 相对来说， [**`任务`**](https://docs.crewai.com/concepts/tasks)抽象定义了智能体需要执行的实际指令。这需要一组不同的参数，封装了 `智能体` 实体，以表明哪些可以参与这个过程，并通过 `.prompt()` 方法计算适当的提示词组件。
> 
> 而且，[**`Crew`**](https://docs.crewai.com/concepts/crews) 抽象同时包含了 `任务` 和 `智能体`，并允许它们通过 `过程` 类以顺序或层次的方式进行通信以实现一系列 `任务`。

**简单来说，CrewAI 抽象帮助您:**
- **定义具有个性、背景和通用目标的智能体。**
- **定义可以由部分智能体以某种方式执行的任务。**
- **定义在不同任务组中工作的智能体团队，并具备多种见证机制。**

这就导致了控制流决策和提示词注入，最终到达您的 LLM 入口，结果响应有助于引导对话和执行环境。

### 查看代码

我们已经讨论了典型的 CrewAI 工作流，下面看看它如何映射到实际代码：

In [ ]:
from crewai import Agent
from crewai.utilities import Prompts

# - You can define one or more CrewAI `Agent`s, which are persona-based agents that communicates with other Agents.
#     - Combined with the `Prompts` utility abstraction, the `task_execution()` method gives you a base prompt for the agent 
#       (which can be added to by other mechanisms later).

# https://docs.crewai.com/concepts/agents#direct-code-definition

teacher_agent = Agent(
    role='Teacher',
    goal="Help students with concerns and make sure they are learning their material well.",
    backstory=(
        "You are a computer science teacher in high school holding office hours, and you have a meeting."
        " This is the middle of the semester, and various students have various discussion topics across your classes."
        " You are having a meeting right now. Please engage with the student."
    ),
    verbose=True,
    memory=True,
    llm=llm,
)

student_agent = Agent(
    role='Student',
    goal="Be a good student while also maintaining personal interests and a healthy social life.",
    backstory=(
        "You are taking Dr. John's intro to algorithms course and are struggling with some of the homework problems."
    ),
    verbose=True,
    memory=True,
    llm=llm,
)

print(Prompts(agent=teacher_agent).task_execution())
print("*" * 64)
print(Prompts(agent=student_agent).task_execution()["prompt"])

In [ ]:
from crewai import Task

# - In contrast, the `Task` abstraction specifies actual directives for your agents to execute on.
#     - This requires a different set of arguments, encapsulates `Agent` entities to communicate which ones can work
#       on the process, and computes an appropriate prompt component via the `.prompt()` method.

# https://docs.crewai.com/concepts/tasks#direct-code-definition-alternative

teacher_task = Task(
    description="Engage in dialog to help the student out.",
    expected_output="Conversational output that is supportive and helpful.",
    tools=[],
    agent=teacher_agent,
    async_execution=False,
    # human_input=True,     # Human-in-the-loop mechanism to correct the agent responses 
)

student_task = Task(
    description="Meet with your teacher to help you understand class material.",
    expected_output="Conversational responses",
    tools=[],
    agent=student_agent,
    async_execution=False,
    # human_input=True,     # Human-in-the-loop mechanism to correct the agent responses 
)

teacher_task.prompt()

In [ ]:
from crewai import Crew, Process

# - And to top it off, the `Crew` abstraction contains both `Task`s and `Agent`s, and allows them to communicate 
#   (via a `Process` class) in a sequential or hierarchical manner to achieve the list of `Tasks`.

chatbot_crew = Crew(
    ## Shift state between teacher and student 4 times (i.e. t->s->t->s->...->s)
    agents=[teacher_agent, student_agent] * 4,
    tasks=[teacher_task, student_task] * 4,
    process=Process.sequential,
    verbose=True,
)

In [ ]:
## Kick off the routine. If there are any {var}s in an agent/task prompt, you can specify inputs={'var': value, ...}
chatbot_crew.kickoff()

<hr><br>

### **反思:** 这比 LangChain 更好吗？

***有时是，有时不是！***
- 对于通用的 LLM 工程，**LangChain 提供的原语灵活性更高。** 课程中不会涵盖的模块和兼容层很多，可以用来制作近乎任意的数据工作流，并且具有隐藏的特性，有利于最终产品化。
- 对于基于个性化系统的智能体应用，**CrewAI 是部署易于指定的智能体组的最简单入口。** 你会发现这里有很多内置假设，并且查看参数列表会揭示各种自定义选项，帮助减轻系统规格的痛苦以及较多的样板代码。
- 对于需要更复杂状态管理系统的自定义应用，**LangGraph 是另一个不错的选择，稍后会用到它。** 这个框架可以轻松深入自定义领域，同时保持核心抽象，但通常需要更好的智能体系统设计理解，因此学习曲线也较高。

从智能体入口的角度来看，可以说 CrewAI 框架更容易上手，因为它让您进入了一些特定的工作流范式。所有这些范式都可以用 LangChain 或 LangGraph 提供的原语组件来实现，但在拥有一个倾向性框架的情况下，您可以轻松接入抽象，从而省去不少麻烦，这显然具有重大价值。因此，当我们发现 CrewAI 能够帮助课程叙述时，就会尽量指出相关的 **CrewAI** 示例解决方案。